Redundancies and symmettries are both lattice like analyses
They are sets of groups elements closed over upon union
It's kind of a cofinite symmettry group?

so group-uf + lattice

free variables is an obvious analysis.

x*0  = 0
fv(x*0) = {x}
fv(0) = {}

meaning = set of terms (possibly infinite)
fvs intersection
ufvs union
The ufvs appear but are allowed to permute to anything not in fvs
But then ufvs can actually permute. so the set is filled out wrt ufvs. fvs and ufvs are very very different.
symmettries operates on fvs (public slots)

To represent    { f(a,b), ... , } we pick a particular value for ufvs and note which are which.


Ok, but then a binger
lam z, e
Not lam is not monotonic because it needs to pick out the terms that don't have z. Or does it? If ufvs can't matter, then even a clash can't matter.

so lam z, e adds z to ufvs



freevars(t)
unhygienicfreevars(t)

The edges aren't the finite groups, it's like the quotient by the current lattice?



https://arxiv.org/pdf/2305.15782 Binding Logic: proofs and models dowek hardin krichner


A EPR union find?
e2(I,J,K) = e1(I,C, K)

I != J  -> e2(I,J) = e1(K, K)

tensor grobner bases. 

We need unifailing completion. Saturation rather than oreintation
Unfailing as Splitting

```
E U {l = r}, R 
----------------------- split
E , R U { l > r | l -> r, l < r, r -> l }


E, R {a < b  | l -> r}    a < b impossible
---------- process1
E, R

E, R {f(x,y) < f(z,w) | l -> r}
-------------------------process2


write term ordering as rule system


      
------------------
  f(X,Y) < g(X,Y)



```





https://egraphs.zulipchat.com/#narrow/channel/556617-rambling/topic/Philip.20Zucker/near/574196637 discussion about whther slotted uf = gorup uf + lattice.

Maybe!

As max says, A group action on the lattice value. That's curious.
That's how it _has_ to work, but I was assuming the action was idempotent.


In [ ]:
class LatticeGroupUF():
   parents : dict[Id, Id]
   group_to_parent : dict[Id, Group]
   lattice_analysis : dict[Root, Lattice]

class BigSymmetySet(Lattice):
   symmetries : list[Group]
   public_slots : list[Slot]
   def join(self, other): # __or__
      symmettres = self.symmettries + other.symmetries
      




# Atomic Union find
Union find with primitives but some 
Nahh doesn't actually make sense
EId int | Prim x | Atom a

In the SMT "model", eids are constants, prims are values, and Atoms

e1(a,b,c) = e2(a,b) canonize to

e1(0,1,2)  = [0->0 1->1] =  e2(0,1)

In summation model, we can factor the sum into every monomial actually as a normal form.
If using summaiton model, lhs and rhs don't share indices. Other inidices are "forall" flexible, not rigid.
What example oculd there be where we have shared rigid alpha invariant atoms on both sides
Exists i, j, a(i,j) = b() . but then the exists is scoped to a single eq.
max, min, lambda, integral, forall, exists, mu, fix, open terms, skolems

EPR union find.
Knuth Bendix Tower idea.


slotted union find does feel like a free variable analysis. differing number of free vars does kind of feel like different scopes, as in 
atoms are kind of between prims and eids in rigidness. prims have axioms to "completely" pin.

nominal union find - trees with atoms.



In [ ]:
type P = tuple[int, ...] # variable pattern
class AtomUF():
    parents : list[list[P, int, P]]


In [14]:
from kdrag.all import *
Atom = smt.DeclareSort("Atom")
T = smt.DeclareSort("T")
LiftInt = kd.Inductive("ListInt")
LiftInt.declare("Atom", ("atom", Atom))
LiftInt.declare("Lit", ("lit", T)) # int?
LiftInt = LiftInt.create()
a,b,c = smt.Consts("a b c", Atom)
f = smt.Function("f", LiftInt, LiftInt, LiftInt)

s = smt.Solver()
va = LiftInt.Atom(a)
def LAtom(name, s):
    a = smt.Const(name, Atom)
    return s.Atom(a) # but then I can't quantify over it.

A = LiftInt.Atom
s.add(smt.ForAll([a,b], a != b, f(A(a),A(b)) == f(A(b),A(a))))
s.check()
s.model()
#s.add(smt.ForAll([a], f(LiftInt.Atom(a)) == LiftInt.Atom(a)))
#s.add(smt.Not(f(f(A(a))) == A(a)))
#s.check()



[f = [else -> Lit(T!val!0)]]

In [ ]:
class PrimUF():
    #prim : list[bool]
    prim : list[Optional[object]]
    parents : lint[int]
    def is_prim(self, eid : int):
        return self.prim[eid] is not None
    def makeset():
        self.parents.append(len(self.parents))
    def set_prim(self, eid):
        eid = self.find(eid)
    def find(self, eid : int):
        while 

    


In [40]:
from dataclasses import dataclass, field
from typing import Callable, NamedTuple

class EId(NamedTuple):
    eid : int

@dataclass
class PrimUF():
    # say which eids are considered prim
    #prim : set[object] = field(default_factory=set)
    #is_prim : Callable[[object],bool] = lambda x: False
    #resolve : 
    parents : dict[object,object] = field(default_factory=dict)
    def find(self, x):
        while x in self.parents and self.parents[x] != x:
            x = self.parents[x]
        return x
    def makeset(self):
        n = len(self.parents)
        while EId(n) in self.parents:
            n += 1
        return EId(n)
    def is_prim(self, x):
        return not isinstance(x, EId)
    def resolve(self, x, y):
        # overlaoding resolve can do things like unification
        if isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
            return tuple(self.union(x1,y1)for x1,y1 in zip(x,y))
        else:
            raise Exception("confict", x, y)
    def union(self, x, y):
        x,y = self.find(x), self.find(y)
        if x != y:
            if self.is_prim(x):
                if self.is_prim(y):
                    return self.resolve(x,y) #raise Exception("conflict", x, y)
                else:
                    x,y = y,x
            self.parents[x] = y
        return y

uf = PrimUF()#is_prim=lambda x: not isinstance(x,int))
x,y,z = [EId(i) for i in range(3)]
uf.union(x,y)
uf.find(x)
uf.union("fred", y)
#uf.union(x, "fred")
uf.find(x)

uf.union((1,z), (1,2))
uf.find(z)

2

A combined primitive and level uf gives us a quantaltuf
Constants should be opaque.

EPrims are not equal to each other.
An eid at a lower level may not union to an eprim at a higher level, because this would be scope escape.

forall xs0 exists ys1 forall xs1 exists ys1 forall xs

If we had a system with explicit existential types, and I do think fancier Haskelly type chekcing has that stuff (Higher rank? Gadts?)

In [ ]:
class EPrim(NamedTuple):
    level : int
    id : object

class EId(NamedTuple):
    level : int
    id : int

# object is like EPrim at largest scope
@dataclass
class QuantUF():
    levels : list[list[EId | EPrim | object]]
    def find(self, x : EId):
        while True:
            level = self.levels[x.level]
            eid = level[x.id]
            if not isinstance(eid, EId) or eid == x:
                return x
    def union(self, x, y):
        x,y = self.find(x), self.find(y)
        if x != y:
            if isinstance(x, EId) and isinstance(y, EId):
            elif not isinstance(x,EId) and not isinstance
            elif isinstance(x,EId):
            if not isinstance(x, EId):
                if isinstance(y, EId):
                    x,y = y,x
                else:
                    return self.resolve(x,y)
            self.parents[x] = y
        return y


    


    

https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/EGRAPHS.20Meeting.202025-03-20.3A.20Slotted.20E-Graphs/near/543099882


"Shouldn’t f(X, Y) = zero imply f(X,X) = zero?

In FO this implication holds. But in slotted and nominal (correct me if I'm wrong) it does not.
And with Zero I meant the number constant, so zeroif you prefer.

By the FO embedding I mentioned above, f(X, Y) = zero would be embedded as ∀X, Y: X != Y -> f(X, Y) = zero,
from which you can't get f(X,X) = zero.
"

"Rudi Schneider: Based on your stratified EPR idea:
Maybe one can faithfully embed a slotted equation (eg. f(X, Y) = g(Y, X)) into multi-sorted FO
simply as ∀ X, Y: Slot,  X != Y -> f(X,Y) = g(Y,X).Rudi Schneider: 
You seemed to think some restriction on the unification occuring during knuth bendix might be necessary?

Yep, another reason to have the "bijective renaming" constraint that I mentioned before, is that otherwise
∀ x, y, a, b:  λx. λy. x = λa. λb. a would imply
∀ x, y, a: λx. λy. x = λa. λa. a which is unsound for typical binders.
"


stratified knuth bendix. If variables only appear in a previous sort compared to the sort that occurs in the `=`

 Also all function symbols go from equal or lower sorts. T0 -> T1 -> T1 is ok, T2 -> T1 is not.

Related to "towers" idea?


Encoding nominal to vampire or eprover


Could i use my knut bendix. Make it unfailing?
Backwards and forwards subsumptions

Just blast all permutations and narrowings?




In [1]:
from kdrag.solvers import kb

## slotted union find
Monomial Tensor Knuth Bendix
a(i,j) = b(j) 
a(i,j) = a(j,i) 
a(j,j) = b(j) 
a(i,j) = c(j,i)

a compact

equlity is symmettricand set like

{a(i,j), b(j)} canonize this set.

It’s very evocative of superposition / knuth bendix, but variables aren’t Unification variables. They are rigid. ??? Is that true?

Maybe I don’t want to canonize atoms out of context of the equality. There is almost no point? If we fuse the two renamings, we get a permutation

The group/groupoid union find

inequality union find arguably maybe only one direction should be allow? Eh. Nah a(i,j) -> b(j)

def union()

We need to look up the canonical form of the atom

subsumptive matching. subsumptive uf = Inequality union find + variant union find

There is an implicit up set in terms of all narrowings.


summation, forall, and constacts
rigid,   substitutable, and constants

It isn't about the slotted atoms being single layer so much as it is about them being stratified. There should be no recursive structure in the thing.

`a(X,Y) = b(Y)`  will always knuth bendix complete because there can't be non trivial overlaps more interesting than those that occur in ground case.

a(i(X)) = b(j(X)) is also fine because the j and such are fine.
Huh. Is that a slotted egraph then? Uhhhh.

`a(e(i,j), e(k,m))`


sum([i,j], a(i,j,m)) == sum([k], b(k,m))
Hmm. No sum as model of rigid doesn't make much sense.

- Lambda
- Integrals
- Einstein - forall, summed, 0

foirall can be lowered to sum. Inequality union find-like

We can enumerate all narrowins and generalizations

stratified knuth bendix is analog of epr (a stratified typing discipline). There is an upper bound on possible overlaps we'll ever find.
We can normalize lowest layer first, then this is stable.

forall x, a(x) = b(x) goes above equality
sum x a (x) = sum y b(y) goes below equality

Analog of the  rename, shape view for sets is (perm, ordlist) pair. Dropping perm is sometimes what you want, to quotient. Sometimes not. Multiset. Hmm. The dedups?

Equality ids and shape ids.
shape -> "perm", shape is particular punning choice were eqid
a(i) = b(j) is tiny two level alpha hash tree 

They just don't feel like permutations.

Some kind of baked in saturation algorithm for a union find dict.

But consider even deduplicating an alpha binary relation.


two kinds of alpha ids.
class ForAllId():
class SumId():

Could have a Sign group  shape -> rename, sign, shape
levi civita, anti symmettry. Kind of nice.

https://einx.readthedocs.io/en/stable/faq/universal.html

Hmm. Kind of intriguing. I think if you have a sort stratified equations (an ordering on your sorts such that all function symbols only go from lower sort to higher sort) knuth bendix ought to work in some sense. This is by analogy with EPR and datalog
There's only so many symbols to work with and overlaps are highly constrained in possibility
Maybe this is exactly EPR?
But like EPR with an equational focus 
effectively propsitional reasoning.  https://microsoft.github.io/ivy/decidability.html
stratified terms are kind of interesting as another fragment besides ground terms that is decidable
But is still like an extensible class of systems
Knuth bendix could still fail, but some kind of unfailing variation ought to terminate / saturate
c(X) = c(Y) seems like a perfectly legit equation that obviously can't be oriented. Also f(X,Y) = f(Y,X)
Since epr is solvable by basically grounding it, maybe this is in some sense ground equations in disguise. Feels a little different though

skoelmization... Could produce a new symbol... Hmm. (slotted) eids?
e(X,Y) = f(e(,), e())
var(X) terminates. var have no equatations.

e(X) = var(X)
or
e(X) <- f(e(X),e(X),e(X))
+
e(X,Y) = e(Z,W). Sloted union find on it's own... Hmm.

THe many equations correspond to the groups at the top. Not so sure that they are groups really though.


forall x, exists e, ...
forall x, lambda e, ...

Could store e(X,Y) = e(Z) stuff to be insta saturated (baked in theory) by rebuilding

What does a "solution" to e(X,Y) = e(Z) look like since it can't be completed?
a ground rewrite system?
a set of terms system?




In [ ]:
%%file /tmp/kb.p
cnf(ax1, a(e1(X,Y)) = e3(X,Y)).
cnf(ax1, a(e1(X,Y)) = e3(X,Y)).
cnf(ax1, a(e1(X,Y)) = e3(X,Y)).
cnf(goal, true != false).

In [33]:
%%file /tmp/kb.p
cnf(ax1, axiom, a(I,J) = b(I,I)).
cnf(ax1, axiom, a(I,J) = a(J,I)).
cnf(ax2, axiom, true != false).

Overwriting /tmp/kb.p


In [34]:
! twee /tmp/kb.p 

Here is the input problem:
  Axiom 1 (ax1): a(X, Y) = b(X, X).
  Axiom 2 (ax1): a(X, Y) = a(Y, X).
  Goal 1 (ax2): true = false.

1. a(X, ?) = b(X, X)
2. a(X, Y) ~> a(X, ?)
3. a(X, ?) ~> a(?, ?)
4. b(X, X) ~> b(?, ?)

Ran out of critical pairs. This means the conjecture is not true.
Here is the final rewrite system:
  a(X, Y) ~> a(X, ?)
  a(X, ?) ~> a(?, ?)
  a(?, ?) -> b(?, ?)
  b(X, X) ~> b(?, ?)

RESULT: Satisfiable (the axioms are consistent).


And then run to knuth bendix totality. Using twee or whatnot. Rudi sent me something like like? No that was during dinner chat right?

In [ ]:
type name = str
type uatom = tuple[str, name, ...]
type shape = tuple[str, int, ...]
type catom = tuple[rename, shape]
def unname(uatom) -> catom:
    env = {}
    inds = []
    for x in uatom[1:]:
        if x not in env:
            env[x] = len(env)
        inds.append(env[x])
    return env, (uatom[0], *inds)
def to_atom(catom) -> uatom:
    rename, shape = catom
    env = {n : x for x,n in rename.items()}
    names = [env[i] for i in shape[1:]]
    return (shape[0], *names)


type perm = dict[int, int]
uf : dict[shape, tuple[perm, shape]] = {}
perms = {}
def find(x : uatom) -> catom:
    rename, shape = unname(x)
    seen = set()
    while shape in uf and shape not in seen:
        perm, shape = uf[shape]
        seen.add(shape)
        rename = {k : perm[v] for k,v in rename.items() if v in perm}
    return to_atom((rename, shape))

def union(x : uatom, y : uatom):
    x = find(x)
    y = find(y)
    if x != y:
        env, sx = unname(x)
        rename, sy = unname(y)
        perm = {}
        for x,i in sorted(rename.items(), key=lambda x:x[1]):
            if x not in env:
                env[x] = len(env)
            perm[i] = env[x]
        uf[sx] = (perm, sy)

def a(*args): return ('a', *args)
def b(*args): return ('b', *args)
def c(*args): return ('c', *args)
i,j,k,l = "i j k l".split()

union(a(i), b(i))
print(uf)
print(find(a(j)))
union(a(j), b(i)) # non trivial self loop... hmm
print(uf)
print(find(a(j)))
uf = {}

union(a(i,j), b(i,j))
print(uf)
union(a(j,i), a(i,j)) 
print(uf)
find(b(l, k))
find(b(k, l))


uf = {}
union(a(i,i), a(i,j))
print(uf)

union(a(i,j) , a(i, k)) # no obviosly oreitable to greater or lesser
 

{('a', 0): ({0: 0}, ('b', 0))}
('b', 'j')
{('a', 0): ({0: 0}, ('b', 0)), ('b', 0): ({0: 1}, ('b', 0))}
('b', 'j')
{('a', 0, 1): ({0: 0, 1: 1}, ('b', 0, 1))}
{('a', 0, 1): ({0: 0, 1: 1}, ('b', 0, 1)), ('b', 0, 1): ({0: 1, 1: 0}, ('b', 0, 1))}
{('a', 0, 0): ({0: 0, 1: 1}, ('a', 0, 1))}


# knuth bendix

a(X) = a(Y) can never be oriented... Hmm.

a(X,Y) = b(X) can only possibly be oriented to b

a(X,Y) = a(Y,X) is commutastivity. It can't be oreitned either


a(i,j) = a(j,i)

a(i) = a(j)
a(i,j) = b(i)


Wait but in the summation model
a(i,j) = b(k)   nothing is shared between.



In [ ]:
{"a" : {(0,1) : ("b", (0,1))}}

In [ ]:
from typing import NamedEnum

class Var(NamedEnum):
    name : object
class SumInd(NamedEnum):
    name : object
class Const(NamedEnum):
    value : int



# mbqi egraph
https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/EGRAPHS.20Meeting.202025-03-20.3A.20Slotted.20E-Graphs/near/543077320

I said that quantifiers of this particular stratified form should terminate.
So z3 in mbqi mode should terminate?
!= constraints gives slotted.



Nominal. What is my model.
Integers + Atoms bolted on.




In [ ]:
from kdrag.all import *

Atom = smt.DeclareSort("Atom")
i,j,k = smt.Consts("i j k", Atom)

T = smt.DeclareSort("T")
Var = smt.Function("Var", Atom, T)

f = smt.Function("f", T, T)

smt.ForAll([i,j], f(Var(i), Var(j)) == f(Var(j), Var(i)))


AT = kd.Inductive("AT")
AT.declare("Atom", ("atom", A))
AT.declare("T", ("t", T))
AT = AT.create()


Z3Exception: b'index out of bounds'

# Modular Hash Cons
https://www.philipzucker.com/toy_bdd/

classes ~ modules

but it isn't type safe.

Make a bdd



In [ ]:
from typing import NamedTuple
import operator

import numpy as np
from dataclasses import dataclass
import dataclasses
import functools

type InLabel = object
type Out = object

class Const(NamedTuple):
    bdd : "BDD"
    val : Out
    def liftA2(self, op, other):
        return other.map(lambda x: op(self.val,x))
    def map(self, f):
        return self.bdd.const(f(self.val))
    # caching achieves a dynamic programming like effect
    @functools.cache
    def __and__(self, x):
        return x.map(lambda y: self.val & y)
    @functools.cache
    def __or__(self, x):
        return x.map(lambda y: self.val | y)
    @functools.cache
    def __invert__(self):
        return self.bdd.const(False if self.val else True)
    def __len__(self):
        return 1 if self.val else 0
    def __eq__(self, other):
        return self is other
    def __hash__(self):
        return hash(id(self))
type BoolExpr = "Const" | "ITE" 

class ITE(NamedTuple):
    bdd : "BDD"
    var : InLabel
    left : BoolExpr
    right : BoolExpr

    def map(self, f):
        left = self.left.map(f)
        right = self.right.map(f)
        if left == right:
            return left
        else:
            return self.bdd.ite(self.var, left, right)

    def __invert__(self):
        return self.map(lambda x: not x)

    def liftA2(self, op, other): # map2
        """
        BDD is basically [Bool] -> Bool. This is an applicative functor
        """
        if isinstance(other, Const):
            return self.map(lambda x: op(x , other.val))
        elif isinstance(other, ITE):
            if other.var == self.var:
                var = self.var
                left = self.left.liftA2(op, other.left)
                right = self.right.liftA2(op, other.right)
            elif other.var > self.var:
                var = other.var
                left = self.liftA2(op, other.left)
                right = self.liftA2(op, other.right)
            elif other.var < self.var:
                var = self.var
                left = self.left.liftA2(op, other)
                right = self.right.liftA2(op, other)
            else:
                raise ValueError("incomparable variables")
            if left == right:
                return left
            else:
                return self.bdd.ite(var, left, right)
        else:
            raise ValueError("unexepcted", other)
    # caches aren't bdd specific.
    @functools.cache
    def __and__(self, other):
        return self.liftA2(operator.and_, other)
    @functools.cache
    def __or__(self, other):
        return self.liftA2(operator.or_, other)
    @functools.cache
    def __xor__(self, other):
        return self.liftA2(operator.xor, other)
    @functools.cache
    def __len__(self):
        return len(self.left) + len(self.right)
    def __eq__(self, other):
        return self is other
    def __hash__(self):
        return hash(id(self))


@dataclass
class BDD():
    #table : dict
    def __init__(self):
        self.table = {}
        self.TRUE = self.const(True)
        self.FALSE = self.const(False)
    def _hashcons(self, x : tuple) -> tuple:
        assert isinstance(x, tuple)
        assert all(t.bdd is self for t in x if isinstance(t, ITE) or isinstance(t, Const))
        key = x[1:] # remove bdd field?  #tuple(id(t) if isinstance(t, self.Node) else t for t in x)
        res = self.table.get(key)
        if res is None:
            self.table[key] = x
            return x
        else:
            return res
    def const(self, v):
        return self._hashcons(Const(self, v))
    def ite(self, c, t, e):
        return self._hashcons(ITE(self, c,t,e))
    def Var(self, x : object):
        return self.ite(x, self.TRUE, self.FALSE)
    def Vars(self, names : str):
        return [self.Var(name) for name in names.split()]

bdd = BDD()
x,y,z = bdd.Vars("x y z")
x & y is (y & x | bdd.FALSE)


True

here
hash ('a', 1, 2)
hash ('a', 1, 2)
there


<>:32: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:32: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
/tmp/ipykernel_579529/1809799814.py:32: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  a is ("a", 1, 2)


array([ITE(var='a0', left=Const(val=True), right=Const(val=False)),
       ITE(var='a1', left=Const(val=True), right=Const(val=False)),
       ITE(var='a2', left=Const(val=True), right=Const(val=False)),
       ITE(var='a3', left=Const(val=True), right=Const(val=False)),
       ITE(var='a4', left=Const(val=True), right=Const(val=False)),
       ITE(var='a5', left=Const(val=True), right=Const(val=False)),
       ITE(var='a6', left=Const(val=True), right=Const(val=False)),
       ITE(var='a7', left=Const(val=True), right=Const(val=False))],
      dtype=object)

ITE(var='x1', left=Const(val=True), right=Const(val=False))

In [ ]:
x1 & x2
print("done")
x1 & x2
len(x1 & x2)

done


3

In [ ]:
        if isinstance(other, bool):
            left = self.left & other
            right = self.right & other
            if left == right:
                return left
            else:
                return ITE(self.var, left, right)
        elif isinstance(other, ITE):
            if other.var == self.var:
                left = self.left.liftA2(op, other.left)
                right = self.right & other.right
                if left == right:
                    return left
                else:
                    return ITE(self.var, left, right)

hash ('a', 1, 2)


(('a', 1, 2),)

In [ ]:
table = {}
def hashcons(x : tuple[...]) -> tuple[...]:
    """
    hashcons returns the actual memory of a canonical tuple assuming that all subterms have already been hashconsed.
    Now one can use `is` to compare canonical terms for equality, which is pointer equality and fast.
    """
    assert isinstance(x, tuple)
    key = tuple(id(t) if isinstance(t, tuple) else t for t in x) # avoids recursion into inner tuples during hashing and equality check internal to table.get
    res = table.get(key)
    if res is None:
        table[key] = x
        return x
    return res